In [49]:
import numpy as np
import pandas as pd
import _pickle as pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('./data/data_clean0.csv')

In [3]:
# present some info
train.head(5)
train.info()
train.shape

,target,0,1,2,3,4,5,6,7,8,...,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990
0,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4992 entries, target to 4990
dtypes: float64(1845), int64(3147)
memory usage: 169.8 MB


In [10]:
# get correlations and keep track of Highly correlated columns for later use.
flds = train.columns
cols = len(flds)
#corr = train.corr().values
imp_corr = []
track_cols = set()

for i in range(cols):
    for j in range(i+1, cols):
        # if both items already written, don't bother to rewrite them again
        if i in track_cols and j in track_cols :
            continue
        if corr[i,j] >= .7 or corr[i,j] <= -.7 :
            imp_corr.append((flds[i], flds[j], corr[i,j]))
            track_cols.add(i)
            track_cols.add(j)
            #print("{:4d}  {:4d}  {:5f}".format(flds[i],flds[j], corr[i,j]))

#np.save('./data/all_correlations.npy', corr)
#with open('./data/imp_correlations.pkl', 'wb') as f:
#    pickle.dump(imp_corr, f)

print('there are %d important correlations with %d total columns'%(len(imp_corr),len(track_cols))) 

there are 1049 important correlations with 1569 total columns


In [11]:
# now group imp correlations into lists so it's easier to use them
grouped_high_corr = []
holder = []
s = -100
for i in imp_corr:
    if s != i[0]:
        if len(holder) > 0 :
            grouped_high_corr.append(holder)
            holder = []
        s = i[0]
        holder.append(s)
    
    holder.append(i[1])
#-----------------------------
print('%d groups'%len(grouped_high_corr))

653 groups


In [50]:
# Instead to drop duplicates, I'll use PCA to reduce each group to one or two columns 
#  without losing information.
PCs_data = np.empty([train.shape[0], 0])
cou = 0
for g in grouped_high_corr:
    group_data = train[g].copy()
    # make sure the data in centered around 0 mean.
    #group_data -= np.mean(group_data, axis=0)
    scaler = MinMaxScaler()
    group_data = scaler.fit_transform(group_data)
    # define PCA classifier
    pca = PCA(n_components=2)
    group_PC = pca.fit_transform(group_data)
    # if second variance_ratio isn't big enough, I'll just drop it.
    if pca.explained_variance_ratio_[1] < 0.5 :
        group_PC = group_PC[:,0].reshape((-1,1))
        cou += 1
    # now add the new data to the large dataframe
    PCs_data = np.concatenate((PCs_data, group_PC), axis=1)
#---------------------------------------------
print('we reduced %d columns to %d columns'%(len(track_cols),PCs_data.shape[1]) )

we reduced 1569 columns to 653 columns


In [62]:
# convert the array into dataframe, set names to the columns
columns = ['pc_corr_%d' % i for i in range(PCs_data.shape[1])]
PCs_df = pd.DataFrame(data=PCs_data, columns=columns)
# drop the columns we used in PCA and then add the 2 dfs together
# I need first to convert track_cols elements from int to str.
cols_to_drop = ['%d' % i for i in track_cols]
train.drop(columns=cols_to_drop, inplace=True)
train = pd.concat((train, PCs_df), axis=1)
# present some info
train.head(5)
train.info()
train.shape

,target,0,2,3,5,6,7,10,11,12,...,pc_corr_643,pc_corr_644,pc_corr_645,pc_corr_646,pc_corr_647,pc_corr_648,pc_corr_649,pc_corr_650,pc_corr_651,pc_corr_652
0,38000000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
1,600000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
2,10000000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
3,2000000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
4,14400000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4076 entries, target to pc_corr_652
dtypes: float64(1931), int64(2145)
memory usage: 138.7 MB


In [63]:
# perfect we finished :D
# save to disk
train.to_csv('./data/data_clean1.csv',index=False)